**Firstly, you should download the model file "pytorch-openpose-master" to your local system, and then modify the relevant path information to ensure the code runs successfully.**

you can find the master model file and the processed videos in  my Google Drive :

**https://drive.google.com/drive/folders/1CgMIg8yLFkGJvWBV_nZk1m35ETliu7Ya?usp=drive_link**

This code snippet imports the `sys` module and appends a specified path to the system path. The path `/Users/wangjiji/Desktop/pytorch-openpose-master/model` should be adjusted to the location of the 'src' directory within the downloaded "pytorch-openpose-master" folder on your local system. This ensures that the Python interpreter can locate and import modules from this directory.

In [2]:
import sys
sys.path.append('/Users/wangjiji/Desktop/pytorch-openpose-master/model')  # Adjust this path to the root where 'src' is located in the downloaded "pytorch-openpose-master" folder


Below is the code that write video to visulise the distances between each skeleton keypoints

**Note the runing time is really long so pls also modify the path of input folder for selected video processing**

In [ ]:
import cv2
import numpy as np
import os
from src.body import Body
from src import util
from tqdm import tqdm
from moviepy.editor import ImageSequenceClip

# Initialize the Body pose model
body_estimation = Body('/Users/wangjiji/Desktop/pytorch-openpose-master/model/body_pose_model.pth')

def extract_keypoints(frame):
    candidate, subset = body_estimation(frame)
    return candidate, subset

def process_frame(frame):
    candidate, subset = extract_keypoints(frame)
    canvas = frame.copy()
    canvas = util.draw_bodypose(canvas, candidate, subset)
    return canvas, candidate, subset

def calculate_distances(candidates):
    distances = []
    for i, candidate1 in enumerate(candidates):
        for j, candidate2 in enumerate(candidates):
            if i < j:
                distance = np.linalg.norm(candidate1[:2] - candidate2[:2])
                distances.append((i, j, distance))
    return distances

def annotate_frame(frame, distances, candidates):
    for (i, j, distance) in distances:
        label = f"Dist: {distance:.2f}"
        x1, y1 = int(candidates[i][0]), int(candidates[i][1])
        x2, y2 = int(candidates[j][0]), int(candidates[j][1])
        cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        mid_x, mid_y = (x1 + x2) // 2, (y1 + y2) // 2
        cv2.putText(frame, label, (mid_x, mid_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return frame

def process_video(video_file, output_file, output_type='video'):
    cap = cv2.VideoCapture(video_file)
    if not cap.isOpened():
        print(f"Cannot open video file {video_file}")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    if output_type == 'video':
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))
    else:
        frames = []

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    progress_bar = tqdm(total=frame_count, desc=f"Processing {os.path.basename(video_file)}", leave=False)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        processed_frame, candidates, subset = process_frame(frame)
        distances = calculate_distances(candidates)
        annotated_frame = annotate_frame(processed_frame, distances, candidates)
        if output_type == 'video':
            out.write(annotated_frame)
        else:
            frames.append(annotated_frame)
        progress_bar.update(1)

    cap.release()
    if output_type == 'video':
        out.release()
    else:
        clip = ImageSequenceClip([cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames], fps=30)
        clip.write_gif(output_file)

    progress_bar.close()
    print(f"Processed video has been written to {output_file}")

def main(input_folder, output_folder, output_type='video'):
    os.makedirs(output_folder, exist_ok=True)
    video_files = [os.path.join(input_folder, f"{i:03}.mp4") for i in range(1, 72)]
    valid_video_files = [f for f in video_files if os.path.exists(f)]
    
    overall_progress = tqdm(total=len(valid_video_files), desc="Overall Progress")

    for video_file in valid_video_files:
        if output_type == 'video':
            output_file = os.path.join(output_folder, f"{os.path.basename(video_file).replace('.mp4', '.mp4')}")
        else:
            output_file = os.path.join(output_folder, f"{os.path.basename(video_file).replace('.mp4', '.gif')}")
        
        process_video(video_file, output_file, output_type)
        overall_progress.update(1)
    
    overall_progress.close()

if __name__ == '__main__':
    input_folder = '/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/Openpose processed'  # Path to the input folder
    output_folder = '/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/Distance Annotated'  # Path to save the processed videos
    os.makedirs(output_folder, exist_ok=True)

    user_input = input("Do you want to generate video or gif? (enter 'video' or 'gif'): ").strip().lower()
    if user_input in ['video', 'gif']:
        output_type = user_input
    else:
        output_type = 'video'  # Default to video if input is invalid

    main(input_folder, output_folder, output_type)

The code is certainly need to be adjusted, maybe we can add some function to detect the major movements between characters...to be modified..